In [ ]:
cd ~/ali.kilinc

In [ ]:
import pymysql
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import inspect
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 120)
from sqlalchemy import create_engine
import datetime
from datas import run_query_ai, create_engine_ai, create_engine_ai2, create_engine_ai3, run_query
inspect.getargspec(run_query_ai)

In [ ]:
database = 'sm'
table_to_use = "sm.ret"

#your query
query = """select * from """ + table_to_use + """ where ;"""

engine = create_engine_ai3(database)
pre_df = run_query(query = query, con = engine)

print(pre_df.head(5))

In [ ]:
#we have a large amount of variables, so we need to handle them manually in preprocessing.
#this csv file keeps variable names, their real dtypes and decide whether they should be discarded or not
var_df = pd.read_csv('varlist_usage_v2.csv', na_values = '', delimiter = ',')
print(var_df.head())
var_df['name'] = var_df['name'].str.lower()

print(var_df.loc[var_df['name'].isin(['a', 'b', 'c']), 'discard'])
var_df.loc[var_df['name'].isin(['d', 'e', 'f']), 'discard'] = 1
print(var_df.loc[var_df['name'].isin(['g', 'h', 'j']), 'discard'])

var_df.loc[var_df['name'].isin(['y']), 'discard'] = 1
var_df.loc[var_df['name'].isin(['z']), 'discard'] = 0

var_df.loc[var_df['name'].isin(['k', 'l']), 'type'] = 'category'

var_drop = list(var_df['name'][var_df['discard']==1])
#print(var_drop)
print(var_df.loc[var_df['name'].isin(['m']), 'type'])

var_df.drop(var_df.loc[var_df['name'].isin(var_drop)==True].index, inplace = True)
var_df.reset_index(drop=True, inplace = True)
print(var_df.head())

pre_df.replace('NA', np.nan, inplace = True)
pre_df.replace('NULL', np.nan, inplace=True)
pre_df.columns = pre_df.columns.str.lower()

pre_df.drop(columns = var_drop, axis = 1, inplace = True)
#var_df['type'] = np.where(war_df['name'].isin([xxxx]), 'category', var_df['type'])
var_df['type'] = np.where(var_df['type'].isin(['int']), 'Int64', var_df['type'])

In [ ]:
#cardinality check, it can also be automated
print(pre_df.nunique().sort_values(ascending=False).head(20))
print(len(pre_df))
#print(variable5.value_counts().sort_values(ascendint=False))

In [ ]:
pre_df_copy = pre_df.copy()
main_df = pre_df.copy()

In [ ]:
main_df_cols = main_df.columns
varlist = list(var_df['name'].str.lower())

var_df.loc[var_df['name'].isin(['k']), 'type'] = 'category'

#changing python dtypes and if they are different than our list, change dtype. Float thing was not necessary, but after so many errors inside pipe, i needed to do it
for i in range(len(main_df_cols)):
    if var_df.loc[i, 'type'] == 'datetime64':
        #main_df[main_df_cols[i]] = pd.to_datetime(main_df[main_df_cols[i]], format = var_df.loc[i, 'timetype'], errors = 'coerce')
        pass
    if var_df.loc[i, 'type'] == 'Int64':
        print(main_df_cols[i])
        print(var_df.loc[i,'type'])
        main_df[main_df_cols[i]] = main_df[main_df_cols[i]].astype('float')
        main_df[main_df_cols[i]] = main_df[main_df_cols[i]].astype(var_df.loc[i,'type'])
    else:
        print(main_df_cols[i])
        print(var_df.loc[i, 'type'])
        main_df[main_df_cols[i]] = main_df[main_df_cols[i]].astype(var_df.loc[i, 'type'])

In [ ]:
#setting a NA threshold and dropping vars according to that. Also fix, empty and duplicate columns are deleted.
main_df.dropna(axis=1, thresh = int(0.40*(main_df.shape[0])), inplace = True)

df_y = main_df[['y']]
main_df.drop(columns = list(df_y.columns), axis = 1, inplace = True)

fix_cols = []
for col in main_df.columns:
    if main_df[col].nunique() == 1:
        fix_cols.append(col)
    else:
        pass
    
emp_cols = []
for col in main_df.columns:
    if main_df[col].count() > 0:
        pass
    else:
        emp_cols.append(col)

print(len(fix_cols))
print(len(emp_cols))

print(main_df.info())

def getDuplicateColumns(df):
    duplicateColumnNames = set()
    duplicateColumnDict = dict()
    for x in range(df.shape[1]):
        col = df.iloc[:,x]
        for y in range(x+1, df.shape[1]):
            otherCol = df.iloc[:,y]
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
                duplicateColumnDict[df.columns[x]] = df.columns[y]
                
    return list(duplicateColumnNames), duplicateColumnDict

dupColNames, dupColDict = getDuplicateColumns(main_df)

for key, value in dupColDict.items():
    print("key = {}, value = {}".format(key,value))
dropcols = list()

dropcols.extend([x for x in fix_cols if x not in dropcols])
dropcols.extend([x for x in emp_cols if x not in dropcols])
dropcols.extend([x for x in dupColNames if x not in dropcols])
print(dropcols)

main_df.drop(columns = dropcols, axis = 1, inplace = True)

var_df = var_df[var_df['name'].isin(list(main_df.columns) + list(df_y.columns)) == True]

In [ ]:
df_y_final = df_y['y']

print(main_df.dtypes.astype(str).value_counts())

date_cols = list(main_df.select_dtypes(include = ['datetime64']).columns)

num_cols = list(main_df.select_dtypes(include = ['float', 'int', 'Int64', 'float64']).columns)
cat_cols = list(main_df.select_dtypes(exclude = ['float', 'int', 'Int64', 'float64', 'datetime64']).columns)

main_df.drop(columns = date_cols, axis = 1, inplace = True)

print(len(num_cols))
print(len(date_cols))
print(len(cat_cols))

main_df1 = main_df.iloc[:50000,]
main_df2 = main_df.iloc[50000:,]
print(main_df1.info())
#!! category dtype do not let you replace or fill np.nan's with new category value, that is why i switched to str type
#divided df into 2 as it was having memory error
for col in cat_cols:
    main_df1[col] = main_df1[col].astype('str')
    main_df1[col].replace(np.nan, 'NULL', inplace = True)
    
for col in cat_cols:
    main_df2[col] = main_df2[col].astype('str')
    main_df2[col].replace(np.nan, 'NULL', inplace = True)
    
main_df = main_df1.append(main_df2, ignore_index = True)

print(main_df.dtypes.astype(str).value_counts())

In [ ]:
print(len(main_df))
print(len(df_y_final))

In [ ]:
from statistics import mean

def get_redundant_pairs(df):
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0,df.shape[1]):
        for j in range(0,i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_triangle(df):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop)
    #print(au_corr)
    au_corr = au_corr.sort_values(ascending=False)
    return au_corr

left_cols = num_cols.copy()
corr_cols = []
elim_rows = []
mean_dict = {}
row = 0
while get_triangle(main_df[left_cols])[0:1].values > 0.6:
    
    #print(len(left_cols))
    sub_df = get_triangle(main_df[left_cols])
    filter_df_1 = [t for t in sub_df.index if (t[0] == sub_df.index[0][0]) or (t[1] == sub_df.index[0][0])]
    filter_df_2 = [t for t in sub_df.index if (t[0] == sub_df.index[0][1]) or (t[1] == sub_df.index[0][1])]
    #print(np.nanmean(sub_df[filter_df_1].values))
    if np.nanmean(sub_df[filter_df_1].values) > np.nanmean(sub_df[filter_df_2].values):
        corr_cols.append(sub_df[0:1].index[0][0])
        left_cols.remove(sub_df[0:1].index[0][0])
    else:
        corr_cols.append(sub_df[0:1].index[0][1])
        left_cols.remove(sub_df[0:1].index[0][1])
    elim_rows.append(sub_df[0:1])
    mean_dict[row]=[(sub_df[0:1].index[0][0], np.nanmean(sub_df[filter_df_1].values)), (sub_df[0:1].index[0][1], np.nanmean(sub_df[filter_df_2].values))]
    row = row + 1
    #print(len(left_cols))

print(corr_cols)
print(elim_rows)
print(mean_dict)

num_cols = [x for x in num_cols if x not in corr_cols]
#print(sub_df[sub_df.apply(lambda x: 'curr_revenue_amt_pre_cr' in x)])

In [ ]:
print(num_cols)
main_df.drop(corr_cols, axis=1, inplace = True)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.inspection import permutation_importance
from eli5.sklearn import PermutationImportance
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.feature_selection import RFECV
from lightgbm import LGBMClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import GenericUnivariateSelect

dt = DecisionTreeRegressor(random_state = 0)
lgb = LGBMClassifier(random_state = 23, max_depth = 6, n_estimators = 300)

main_df[cat_cols].replace(np.nan, 'NULL', inplace=True)
orden = OrdinalEncoder()
main_df[cat_cols] = orden.fit_transform(main_df[cat_cols])
main_df[num_cols] = main_df[num_cols].fillna(main_df[num_cols].median())

print(main_df.head())
print(df_y_final.head())
pi = PermutationImportance(dt, cv=3,random_state = 0)


#rfecv has a problem with pipeline recognition
cat_transformer = Pipeline(steps = [('enc', OrdinalEncoder())])
num_transformer = Pipeline(steps = [('sc1', MinMaxScaler(feature_range = (0,1)))])
preprocessor = ColumnTransformer(transformers = [('cat', 'passthrough', cat_cols), ('num', 'passthrough', num_cols)], remainder = 'passthrough')

all_pipe = Pipeline(steps = [('prep', preprocessor), ('est', lgb)])

rfe = RFECV(estimator = pi, cv=3, min_features_to_select = 80, scoring = 'neg_mean_squared_error', step = 2, n_jobs =4)
rfe.fit(main_df, df_y_final)

"""
mutual_inf = mutual_info_classif(main_df, df_y_final, random_state = 10)
generic = GenericUnivariateSelect(score_func = mutual_info_classif, mode = 'k_best', param=100)
main_df_trans = generic.fit_transform(main_df, df_y_final)
print(main_df['var8'][main_df['var8'].isnull()])
"""

In [ ]:
main_df[cat_cols] = orden.inverse_transform(main_df[cat_cols])
cols_left = main_df.columns[rfe.get_support()].values
df_left = pd.DataFrame(main_df, columns = cols_left)
print(cols_left)
print(len(cols_left))
num_vars = []
cat_vars = []
for col in cols_left:
    if col in cat_cols:
        df_left[col] = df_left[col].astype('object')
        cat_vars.append(col)
    else:
        pass
    
for col in cols_left:
    if col in num_cols:
        df_left[col] = df_left[col].astype('float')
        num_vars.append(col)
    else:
        pass
    
print(num_vars)
print(cat_vars)

In [ ]:
def col_ind_get(prep, all_cols, num_cols, cat_cols, enc_cols):
    
    pre_all_cols = []
    pre_cat_cols = []
    pre_cat_ind = []
    
    cat_cols = list(set(cat_cols) - set(enc_cols))
    
    for i in range(len(prep.transformers)):
        
        pre_all_cols.extend(prep.transformers[i][2])
        
    pre_cat_cols = [item for item in pre_all_cols if item in cat_cols]
    pre_cat_ind = [i for i in range(len(pre_all_cols)) if pre_all_cols[i] in cat_cols]
    pre_num_cols = [item for item in pre_all_cols if item in num_cols]
    pre_num_ind = [i for i in range(len(pre_all_cols)) if pre_all_cols[i] in num_cols]
    
    return pre_all_cols, pre_cat_cols, pre_cat_ind, pre_num_cols, pre_num_ind

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor, Pool, FeaturesData

print(cat_vars)
print(num_vars)
ordered_cols = cat_vars + num_vars

df_y_final = df_y['y']

df_left2 = df_left.reindex(columns = ordered_cols)

test_list = []

for col in cat_vars:
    df_left2[col]=df_left2[col].astype('str').str.replace('.0','')

for col in cat_vars:
    #print(col)
    cont = df_left2[col][df_left2[col].astype('str').str.contains('35.0')]
    #print(cont)
    if len(cont) > 0:
        test_list.append(cont)


feature_selection = 1
enc_vars = []

if feature_selection == 1:
    models = dict()
    paramscat = {
        #'est__depth': np.linspace(8,8,1,endpoint=True),
        'est__depth': [8],
        'est__iterations': [600],
        'est__learning_rate': [0.01],
        #'est__bagging_temperature': [10,20],
        #'est__border_count': [75],
        #'est__loss_function': ['MultiClass'],
        #'est__eval_metric': ['AUC']
    }

models['catb'] = [CatBoostRegressor(verbose = 0), paramscat]

for key, value in models.items():
    
    scorer = ['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2']
    
    numeric_transformer = Pipeline(steps = [('scl', MinMaxScaler(feature_range = (0,1)))])
    preprocessor = ColumnTransformer(transformers = [('cat', 'passthrough', cat_vars),
                                                    ('num', 'passthrough', num_vars)], remainder = 'passthrough')
    
    all_pipe = Pipeline(steps = [('prep', preprocessor), ('est', value[0])])
    
    pre_all_cols, pre_cat_cols, pre_cat_ind, pre_num_cols, pre_num_ind = col_ind_get(preprocessor, df_left2.columns, num_vars, cat_vars, enc_vars)
    search_space = value[1]
    print(pre_all_cols)
    print(pre_cat_cols)
    print(pre_cat_ind)
    print(pre_num_cols)
    print(pre_num_ind)
    print(list(df_left2.columns))
    print(df_left2.columns)
    print(df_left2.head(2))
    print(df_y_final.head(5))
    grid_search = GridSearchCV(all_pipe, search_space, cv= 5, verbose = 1, refit = 'neg_mean_squared_error', scoring = scorer, return_train_score = True, n_jobs=4)
    grid_search.fit(df_left2, y=df_y_final,est__cat_features=pre_cat_ind)
    
ind = grid_search.best_index_
print("model = {}".format(key))
print("train_mse = {}, test_mse = {}".format(grid_search.cv_results_['mean_train_neg_mean_squared_error'][ind], grid_search.cv_results_['mean_test_neg_mean_squared_error'][ind]))
print("train_mae = {}, test_mae = {}".format(grid_search.cv_results_['mean_train_neg_mean_absolute_error'][ind], grid_search.cv_results_['mean_test_neg_mean_absolute_error'][ind]))
print("train_r2 = {}, test_r2 = {}".format(grid_search.cv_results_['mean_train_r2'][ind], grid_search.cv_results_['mean_test_r2'][ind]))
print("avg_fit_time = {}".format(grid_search.cv_results_['mean_fit_time'][ind]))
print("best_params = {}".format(grid_search.best_params_))
print("best_estimator = {}".format(grid_search.best_estimator_.named_steps['est']))

best_params = dict()
best_params[key] = grid_search.best_params_

best_results = dict()
best_results[key] = [grid_search.cv_results_['mean_test_neg_mean_squared_error'][ind], grid_search.cv_results_['mean_test_neg_mean_absolute_error'][ind], grid_search.cv_results_['mean_test_r2'][ind]]

importances = grid_search.best_estimator_.named_steps['est'].feature_importances_
importances = [float(i) for i in importances]

print(pre_all_cols)
print(importances)
print(list(zip(pre_all_cols, importances)))

In [ ]:
imp_df = pd.DataFrame(columns = ['col', 'imp'])
imp_df['col'] = pre_all_cols
imp_df['imp'] = importances
imp_df['dtype'] = np.where(imp_df['col'].isin(num_vars), 'num','cat')
imp_df.sort_values(by = ['imp'], ascending = False, inplace = True, ignore_index = True)
print(imp_df.head(5))

top_imp_df = imp_df.iloc[0:60,]

print(top_imp_df)

In [ ]:
import pandas_profiling as pp

filtered_df = df_left2[list(top_imp_df['col'])]

df_y_report = pd.DataFrame(df_y_final, columns = ['y'])
print(df_y_report)
report_df = pd.concat([filtered_df, df_y_report], axis = 1)
report_df = pd.DataFrame(report_df)
report_df['y'] = report_df['y'].astype('object')
print(report_df.info())

report.to_file('report_new.html')